# Machine Learning Implementation

## Structure

#### 1. Helper Functions

There are 2 helper funcitons used throughout this section:

* _return_df_pred_summaries_: returns the predicted summaries given the fixed number of sentences required 
* _calc_rouge_scores_: calculates average Rouge scores across multiple predicted and gold summary pairs

#### 2. Supervised Learning Using Only Embedding Information

* Logistic Regression Models
    * Defaul model
    * Balanced class weights
    * Elastic Net with hyperparamter tuning <br><br>
    
* Neural Net Models
    * Single layer dense network with 25 neurons
    * Single layer dense network with 50 neurons
    * Single layer dense network with 75 neurons
    * Double layer dense dense network with 25 and 25 neurons
    * Double layer dense dense network with 25 and 50 neurons
    * Double layer dense dense network with 50 and 50 neurons<br><br>
    
* TextRank
    * TextRank ranking matrix per article
    * TextRank rouge score


#### 3. Supervised Learning Including Sequential Information

* Logistic Regression Models
    * Defaul model
    * Balanced class weights <br><br>
    
* Long Short Term Memory Models
    * Single layer unidirectional LSTM with 25 neurons
    * Single layer unidirectional LSTM with 50 neurons
    * Single layer bidirectional LSTM with 25 neurons
    * Single layer bidirectional LSTM with 50 neurons<br><br>

* LEDE3
 

## 1. Helper Functions

return_df_pred_summaries_: returns the predicted summaries given the fixed number of sentences required 

In [ ]:
'''functions.py'''

###Sub-function used in return_pred_summaries

def return_greater_than_min_num(arr, thresh=0.5, min_num=1, fix_num_flag=False, fix_num=3):
    
    '''returns top sentences by index numbers in ascending format and according to input
    specifications
    '''
    #want fixed number sentences?
    if fix_num_flag == True:
        idx = np.argsort(arr)[-fix_num:]
        
    #return above model threshold only    
    else:
        idx_prelim = np.where(arr>= thresh)
        
        #filter for minimum number required
        if idx_prelim[0].shape[0] <= min_num:
            idx = np.argsort(arr)[-min_num:]
        else:
            idx = idx_prelim
            
    #return in ascending order
    return sorted(idx)


###Main helper function    
def return_df_pred_summaries( Xy_doc_label, y_pred, df_text, thresh, min_num,
                             return_all=False, fix_num_flag=False, fix_num=3):
    
    '''return list of predicted summaries and additional information if required
    and according to inout specifications'''
    
    #Wrangle to doc label and flattened array of predictions for each article
    df_label_pred = pd.DataFrame({'doc_label': Xy_doc_label.flatten(),
                                                 'y_pred': y_pred.flatten()}) 
    df_label_pred = df_label_pred.groupby('doc_label').agg(list) 

    df_label_pred = df_label_pred.applymap(lambda x: np.array(x))

    #subfunction to lambda
    f = lambda arr: return_greater_than_min_num(arr, thresh=thresh, 
                                    min_num=min_num,fix_num_flag = fix_num_flag, 
                                                            fix_num=fix_num)
    #get sorted index sentence numbers to include in article
    df_label_pred = df_label_pred.applymap(f) 

    #Return predicted summary
          #index is doc label
    df_doc = df_text[df_label_pred.index]     
    
          # return article sentences as list
    pred_summaries = [np.array(df_doc.iloc[j])       
                               [df_label_pred.iloc[j][0]].tolist()                      
                                          for j in range(len(df_label_pred))]
          #join into summary as single string
    pred_summaries = [summ_list if type(summ_list) == str else   
                      ' '.join(summ_list) for summ_list in pred_summaries]  
    
    if return_all == True:
        answer = df_label_pred.values, df_label_pred.index, pred_summaries
    else:
        answer = pred_summaries
    
    return answer

calc_rouge_scores: calculates average Rouge scores across multiple predicted and gold summary pairs

## 2. Supervised Learning Using Only Embedding Information

 ### 2.1 Logistic Regression Models

__2.1.1__ General Implementation

In [ ]:
'''
logreg_default_embeddings_only.py
'''

import pickle
import pandas as pd
from functions import return_df_pred_summaries
from functions import calc_rouge_scores
from sklearn.linear_model import LogisticRegression

from datetime import datetime as dt

from sklearn.metrics import confusion_matrix

input_filename = 'train_test_set20_embeddings_only.pickle'

output_file = 'XXX.pickle'


t1 = dt.now()
print(t1)

data_dict = pd.read_pickle(input_filename)

#Specify model inputs: df, X, y, doc_labels
df = data_dict['df_original']
train_test_set = data_dict['train_test_sets']
#Specify train-test_data for validation        
Xy_doc_label_train = train_test_set[0][0]
Xy_doc_label_test = train_test_set[0][1]
X_train = train_test_set[0][2]
X_test = train_test_set[0][3]
y_train = train_test_set[0][4]
y_test = train_test_set[0][5]

#Define Model
model = LogisticRegression(random_state=42)
        # LogisticRegression(class_weight='balanced', random_state=42)
        # LogisticRegression(solver='saga', penalty='elasticnet',
                           #l1_ratio=0.25, C=0.5, random_state=42)
#Fit model
model.fit(X_train,y_train)
#Predict Model
y_pred = model.predict_proba(X_test)
    
#Convert to binary predictions
y_pred_bin = (y_pred >=0.5)*1

cm = confusion_matrix(y_test, y_pred_bin[:,1], labels=[0,1])

#Return predicted summaries
idx, doc_index, pred_summaries = return_df_pred_summaries(Xy_doc_label_test, 
                                y_pred[:,1], df.text_clean, thresh=0.5, min_num=1, 
                                return_all = True, fix_num_flag=True, fix_num=3)

#Match with gold summaries
df_gold = df.summary_clean[doc_index]
gold_summaries = [' '.join(df_gold .iloc[j]) for j in range(len(pred_summaries))]
summaries_comp = tuple(zip(pred_summaries, gold_summaries))

scores = calc_rouge_scores(pred_summaries, gold_summaries, 
                                  keys=['rouge1', 'rougeL'], use_stemmer=True)




results_dict ={'conf_matrix': cm, 'summaries_comp': summaries_comp,
               'sent_index_number': idx, 'Rouge': scores}

with open(output_file, 'wb') as handle:                                     
    pickle.dump(results_dict, handle)

print(scores)

t2 = dt.now()
print(t2)
print(t2-t1)

__2.1.2__ Elastic Net Hyperparameter Tuning

In [ ]:
'''
logreg_embeddings_only_gridsearch.py
'''
import pickle
import pandas as pd
from functions import return_df_pred_summaries
from functions import calc_rouge_scores
from sklearn.linear_model import LogisticRegression

from datetime import datetime as dt

input_filename = 'train_test_set20_embeddings_only.pickle'             

output_file = 'cv_results_top_3_logreg_elasticnet_gridsearch.pickle'


t1 = dt.now()
print(t1)

data_dict = pd.read_pickle(input_filename)

#Specify model inputs: df, X, y, doc_labels
df = data_dict['df_original']
train_test_set = data_dict['train_test_sets']
#Specify train-test_data for validation        
Xy_doc_label_train = train_test_set[0][0]
Xy_doc_label_test = train_test_set[0][1]
X_train = train_test_set[0][2]
X_test = train_test_set[0][3]
y_train = train_test_set[0][4]
y_test = train_test_set[0][5]

#Define Model
l1_ratio_list = [0.25, 0.5, 0.75, 1]
C_list = [0.25, 0.5, 1, 2, 4]

rouge1_f1_list = []
params_list = []

for l1_ratio in l1_ratio_list:
    for C in C_list:
        
        model = LogisticRegression(solver='saga', penalty='elasticnet', l1_ratio=l1_ratio,
                           C=C, random_state=42)
        #Fit model
        model.fit(X_train,y_train)
        #Predict Model
        y_pred = model.predict_proba(X_test)
    
        #Convert to binary predictions
        y_pred_bin = (y_pred >=0.5)*1
   
        #Return predicted summaries
        idx, doc_index, pred_summaries = return_df_pred_summaries(Xy_doc_label_test, 
                                y_pred[:,1], df.text_clean, thresh=0.5, min_num=1, 
                                return_all = True, fix_num_flag=True, fix_num=3)

        #Match with gold summaries
        df_gold = df.summary_clean[doc_index]
        gold_summaries = [' '.join(df_gold .iloc[j]) for j in range(len(pred_summaries))]
        summaries_comp = tuple(zip(pred_summaries, gold_summaries))

        scores = calc_rouge_scores(pred_summaries, gold_summaries, 
                                  keys=['rouge1', 'rougeL'], use_stemmer=True)
        f1 = scores['rouge1']['f1']
        rouge1_f1_list.append(f1)
        
        params = (l1_ratio, C)
        params_list.append(params)
        
results_dict = {'params': params_list, 'scores': rouge1_f1_list}        

with open(output_file, 'wb') as handle:                                     
    pickle.dump(results_dict, handle)

print(scores)

t2 = dt.now()
print(t2)
print(t2-t1)

 ### 2.2 Neural Net Models
 
General Implementation

In [ ]:
'''neural_network_embeddings_only'''
import pickle
import pandas as pd
from functions import return_df_pred_summaries
from functions import calc_rouge_scores
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.metrics import confusion_matrix

from datetime import datetime as dt

input_filename = 'train_test_set20_embeddings_only.pickle'

output_file = 'XXX.pickle'

t1 = dt.now()
print(t1)

data_dict = pd.read_pickle(input_filename)

#Specify model inputs: df, X, y, doc_labels
df = data_dict['df_original']
train_test_set = data_dict['train_test_sets']
#Specify train-test_data for validation        
Xy_doc_label_train = train_test_set[0][0]
Xy_doc_label_test = train_test_set[0][1]
X_train = train_test_set[0][2]
X_test = train_test_set[0][3]
y_train = train_test_set[0][4]
y_test = train_test_set[0][5]

#class_weights for imbalanced data
pos_w = int(y_train.shape[0] / sum(y_train==1)[0])
weight_dict = {0:1, 1: pos_w/2}
   
#Define Model
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model = Sequential()
model.add(Dense(25, input_dim=1536, activation='relu'))
#model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
    
#Compile Model
model.compile(loss='binary_crossentropy', optimizer='adam', 
metrics=[tf.keras.metrics.SensitivityAtSpecificity(0.5, num_thresholds=1),
             tf.keras.metrics.SpecificityAtSensitivity(0.5, num_thresholds=1)])
#Fit Model
history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                          callbacks=[callback], class_weight=weight_dict) #class_weight=weight_dict
#Predict Model
y_pred = model.predict(X_test)
    
#Convert to binary predictions
y_pred_bin = (y_pred >=0.5)*1

cm = confusion_matrix(y_test, y_pred_bin, labels=[0,1])


#Return predicted summaries
idx, doc_index, pred_summaries = return_df_pred_summaries(Xy_doc_label_test, 
                                y_pred, df.text_clean, thresh=0.5, min_num=1, 
                                return_all = True, fix_num_flag=True, fix_num=3)

#pred_summaries = [' '.join(df.text[doc_index].iloc[j][:3]) for j in range(len(idx))]

#Match with gold summaries
df_gold = df.summary_clean[doc_index]
gold_summaries = [' '.join(df_gold .iloc[j]) for j in range(len(pred_summaries))]
summaries_comp = tuple(zip(pred_summaries, gold_summaries))

scores = calc_rouge_scores(pred_summaries, gold_summaries, 
                                  keys=['rouge1', 'rougeL'], use_stemmer=True)



results_dict ={'conf_matrix': cm, 'summaries_comp': summaries_comp,
               'sent_index_number': idx, 'Rouge': scores, 'mod_summary': model.summary()}

with open(output_file, 'wb') as handle:                                     
    pickle.dump(results_dict, handle)

print(scores)

t2 = dt.now()
print(t2)
print(t2-t1)

 ### 2.3 TextRank
 
 __2.3.1__ Calculate ranking matrix per article

In [ ]:
'''textrank_ranking_matrix.py'''
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from datetime import datetime as dt

t1 = dt.now()
print(t1)

output_file = 'train_stats_dict_processed_extr_final_5000_inc_pagerank.pickle' 
input_file = 'train_stats_dict_processed_extr_final_5000_.pickle' 
data = pd.read_pickle(input_file )

#Select sentence embeddings only and match to doc label
df_embed = data['df_X'].loc[:,'Sent_BERT_D_0': 'Sent_BERT_D_767']
df_doc_label = pd.DataFrame(data['Xy_doc_label_array'],columns=['doc_label'])
df = pd.concat([df_doc_label, df_embed], axis=1)

#loop through articles (docs)
pagerank_scores_list=[]
error_list = []
doc_num = np.max(data['Xy_doc_label_array']) 
for j in range(doc_num+1):
    
    #calculate cosine similiarity matrix 
    df_doc = df [df.doc_label == j].iloc[:,2:]
    n = df_doc.shape[0]
    cos_matrix = cosine_similarity(df_doc, df_doc)
    f = np.vectorize(lambda x: 0 if x == 1 else 1)
    not_eye = f(np.eye(n,n))
    cos_matrix = cos_matrix * not_eye
    
    #Convert to nx graph
    graph = nx.from_numpy_array(cos_matrix)
    
    #Calculate sentence scores and record error docs
    try:
        scores_arr = np.array(list(nx.pagerank(graph, max_iter=500).values()))
    except:
        scores_arr = np.nan
        error_list.append(j)
   
    pagerank_scores_list.append(scores_arr)
    
pagerank_scores_arr = np.array(pagerank_scores_list)

#store in primary dictionary
data.update({'textrank_scores_arr_per_doc':pagerank_scores_arr })

#save to pickle
with open(output_file, 'wb') as handle:                                     
    pickle.dump(data, handle)

t2=dt.now()
print(t2)
print(t2-t1)

#runtime 4mins50sec for 5000 docs / 29 errors

__2.3.2__ Calculate TextRank Rouge score

In [ ]:
'''textrank_rouge_score.py'''

import pandas as pd
import numpy as np
import pickle
from functions import calc_rouge_scores


input_textrank = 'train_stats_dict_processed_extr_final_5000_inc_pagerank.pickle'
input_test_labels = 'train_test_set20_embeddings_only.pickle'
output_file = 'cv_results_textrank.pickle'

data = pd.read_pickle(input_textrank )

test_labels = pd.read_pickle(input_test_labels)
test_labels = set(test_labels['train_test_sets'][0][1].flatten())

#original df with columns including article / summary text
df = data['df_original']
#add pagerank scores to df
df['textrank_scores'] = data['textrank_scores_arr_per_doc']
#filter for test set
df = df[df.index.isin(test_labels)]
#drop where textrank had errors
df = df.dropna()

#pick top3 sentence by textrank score
df['idx'] = df['textrank_scores'].apply(lambda x: sorted(np.argsort(x)[-3:])).values
idx_arr = df['idx'].values

#convert list of sentences to string for each predicted summary
pred_summaries = [' '.join(np.array(df.text_clean.iloc[j])[idx_arr[j]].tolist())
                  for j in range(len(idx_arr))]

#convert cleaned gold summarysentence lists to string for each summary
df_gold = df.summary_clean
gold_summaries = [' '.join(df_gold .iloc[j]) for j in range(len(pred_summaries))]

#zip each predicted / gold summary pair together and store in another tuple
summaries_comp = tuple(zip(pred_summaries, gold_summaries))

#calculate rouge scores
scores = calc_rouge_scores(pred_summaries, gold_summaries, 
                                  keys=['rouge1', 'rougeL'], use_stemmer=True)
#store results in dict
results_dict = {'Rouge': scores, 'doc_labels': df.index.tolist(),
                'summaries_comp': summaries_comp}
#add to primary dict
data.update(results_dict)

#save to pickle
with open(output_file, 'wb') as handle:                                     
    pickle.dump(data, handle)


## 3. Supervised Learning Including Sequential Information

 ### 3.1 Logistic Regression Models
 
 __3.1.1__ General Implementation

In [ ]:
'''logreg_embeddings_num_sent.py'''
import pickle
import pandas as pd
from functions import return_df_pred_summaries
from functions import calc_rouge_scores
from sklearn.linear_model import LogisticRegression

from datetime import datetime as dt

from sklearn.metrics import confusion_matrix

input_filename = 'train_test_set20_embeddings_sent_num.pickle'     

output_file = 'XXX.pickle'

t1 = dt.now()
print(t1)

data_dict = pd.read_pickle(input_filename)

#Specify model inputs: df, X, y, doc_labels
df = data_dict['df_original']
train_test_set = data_dict['train_test_sets']
#Specify train-test_data for validation        
Xy_doc_label_train = train_test_set[0][0]
Xy_doc_label_test = train_test_set[0][1]
X_train = train_test_set[0][2]
X_test = train_test_set[0][3]
y_train = train_test_set[0][4]
y_test = train_test_set[0][5]

#Define Model
model = LogisticRegression(random_state=42)
        # LogisticRegression(class_weight='balanced', random_state=42)
#Fit model
model.fit(X_train,y_train)
#Predict Model
y_pred = model.predict_proba(X_test)
    
#Convert to binary predictions
y_pred_bin = (y_pred >=0.5)*1

cm = confusion_matrix(y_test, y_pred_bin[:,1], labels=[0,1])

   
#Return predicted summaries
idx, doc_index, pred_summaries = return_df_pred_summaries(Xy_doc_label_test, 
                                y_pred[:,1], df.text_clean, thresh=0.5, min_num=1, 
                                return_all = True, fix_num_flag=True, fix_num=3)


#Match with gold summaries
df_gold = df.summary_clean[doc_index]
gold_summaries = [' '.join(df_gold .iloc[j]) for j in range(len(pred_summaries))]
summaries_comp = tuple(zip(pred_summaries, gold_summaries))

scores = calc_rouge_scores(pred_summaries, gold_summaries, 
                                  keys=['rouge1', 'rougeL'], use_stemmer=True)


results_dict ={'conf_matrix': cm, 'summaries_comp': summaries_comp,
               'sent_index_number': idx, 'Rouge': scores}

with open(output_file, 'wb') as handle:                                     
    pickle.dump(results_dict, handle)

print(scores)

t2 = dt.now()
print(t2)
print(t2-t1)


 ### 3.2 Long Short Term Memory
 
General Implementation

In [ ]:
'''lstm1.py'''
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import TimeDistributed
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from functions import calc_rouge_scores
from keras.layers import Bidirectional


input_filename = 'train_test_set20_embeddings_only.pickle'
output_file = 'XXX.pickle'

data_dict = pd.read_pickle(input_filename)

df = data_dict['df_original']

#step 1: process data for ltsm input

#convert to numpy array
to_array = lambda x: np.array(x)
df.text_embedding = df.text_embedding.apply(to_array)
df.labels= df.labels.apply(to_array)
df.text_embedding = df.text_embedding.apply(lambda x: x.reshape(1, x.shape[0],x.shape[1]))
df.labels = df.labels.apply(lambda x: x.reshape(1, len(x),1))

#train_test split
train_doc_labels = set(data_dict['train_test_sets'][0][0].flatten())
mask_train = np.array([x in train_doc_labels for x in df.index]) 

X_train = df.text_embedding[mask_train].tolist()
y_train = df.labels[mask_train].tolist()

X_test = df.text_embedding[~mask_train].tolist()
y_test = df.labels[~mask_train].tolist()


# define LSTM
model = Sequential()

model.add(LSTM(25, input_shape=(None, 768), return_sequences=True, dropout=0))
#model.add(LSTM(25, input_shape=(None, 768), return_sequences=True, dropout=0))

#model.add(Bidirectional(LSTM(50, return_sequences=True, dropout=0), input_shape=(None, 768)))
#model.add(Bidirectional(LSTM(50, return_sequences=True, dropout=0), input_shape=(None, 768)))

model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=[tf.keras.metrics.SensitivityAtSpecificity(0.5, num_thresholds=1)])


# train LSTM
training_loss =[]
training_metric = []
for j in range(len(X_train)):   
    X, y = X_train[j], y_train[j]
    history = model.fit(X, y, epochs=1, batch_size=1)
    training_loss.append(history.history['loss'])
    
# evaluate LSTM
y_pred_list =[]
idx_list=[]
for j in range(len(X_test)):
    X= X_test[j]
    y_pred = model.predict(X, verbose=0)
    idx = np.argsort(y_pred[0].flatten())[-3:]
    idx = sorted(idx)
    y_pred_list.append(y_pred)
    idx_list.append(idx)

    
#retrieve summary pairs
doc_index = df.index[~mask_train]
pred_summaries = [' '.join(np.array(df.text_clean[doc_index].iloc[j])[np.array(idx_list[j])].tolist()) 
                  for j in range(len(idx_list))]
df_gold = df.summary_clean[doc_index]
gold_summaries = [' '.join(df_gold .iloc[j]) for j in range(len(pred_summaries))]
summaries_comp = tuple(zip(pred_summaries, gold_summaries))


#calculate rouge score
scores = calc_rouge_scores(pred_summaries, gold_summaries, 
                                  keys=['rouge1', 'rougeL'], use_stemmer=True)

results_dict ={'summaries_comp': summaries_comp,
               'sent_index_number': idx, 'Rouge': scores, 'mod_summary': model.summary()}

with open(output_file, 'wb') as handle:                                     
    pickle.dump(results_dict, handle)

 ### 3.2 LEDE3
 
Implementation

In [ ]:
'''LEDE3.py'''
import pickle
import pandas as pd
import numpy as np
from functions import calc_rouge_scores

from datetime import datetime as dt

input_filename = 'train_test_set20_embeddings_sent_num.pickle'             

output_file = 'cv_results_LEDE3.pickle'

t1 = dt.now()
print(t1)

data_dict = pd.read_pickle(input_filename)

#Specify model inputs: df, X, y, doc_labels
df = data_dict['df_original']

#Specify train-test_data for validation        
train_doc_labels = set(data_dict['train_test_sets'][0][0].flatten())
mask_train = np.array([x in train_doc_labels for x in df.index]) 
mask_test = ~mask_train

#Return predicted summary
df_doc = df.text_clean[mask_test]
    
pred_summaries = [np.array(df_doc.iloc[j])[:3]
                                          for j in range(len(df_doc))]

pred_summaries = [summ_list if type(summ_list) == str else 
                      ' '.join(summ_list) for summ_list in pred_summaries]

#Match with gold summaries
df_gold = df.summary_clean[mask_test]
gold_summaries = [' '.join(df_gold .iloc[j]) for j in range(len(pred_summaries))]
summaries_comp = tuple(zip(pred_summaries, gold_summaries))

scores = calc_rouge_scores(pred_summaries, gold_summaries, 
                                  keys=['rouge1', 'rougeL'], use_stemmer=True)

results_dict ={'summaries_comp': summaries_comp,'Rouge': scores}

with open(output_file, 'wb') as handle:                                     
    pickle.dump(results_dict, handle)

print(scores)

t2 = dt.now()
print(t2)
print(t2-t1)